[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RajGajjar-01/LangChain-LangGraph/blob/main/Langgraph/16_basic_MCP_client.ipynb)

In [ ]:
# Install dependencies if running in Google Colab
import sys
if 'google.colab' in sys.modules:
    !pip install -U langgraph langchain-google-genai langchain-mcp-adapters python-dotenv

## **Model Context Protocol (MCP) Client in LangGraph**

<div class="premium-card">
    This notebook demonstrates how to build a <b>Model Context Protocol (MCP)</b> client using LangGraph. MCP allows agents to interact with external tools and data sources through a standardized protocol. Here, we use a filesystem MCP server to give the agent file management capabilities.
</div>

### **1. Imports and environment setup**

In [1]:
import os
import asyncio
import subprocess, textwrap

from dotenv import load_dotenv
from typing import TypedDict

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

from langchain_mcp_adapters.client import MultiServerMCPClient

### **2. Sandbox Directory Setup**

We create a dedicated directory for the agent to safely perform file operations.

In [2]:
SANDBOX_DIR = os.path.abspath("sandbox_dir")
os.makedirs(SANDBOX_DIR, exist_ok=True)
print("Sandbox", SANDBOX_DIR)

Sandbox /home/rajgajjar04/Learnings/Langchain_Langgraph/Langgraph/sandbox_dir


### **3. Connect to MCP Server**

We initialize the <code>MultiServerMCPClient</code> to connect to a filesystem server using <code>npx</code>.

In [4]:
async def test_mcp_list_tools():
    client = MultiServerMCPClient(
        {
            "filesystem": {
                "command": "npx",
                "args": [
                    "-y",
                    "@modelcontextprotocol/server-filesystem",
                    SANDBOX_DIR
                ],
                "transport": "stdio",
            }
        }
    )

    tools = await client.get_tools()
    for t in tools:
        print("-", t.name)

await test_mcp_list_tools()

- read_file
- read_text_file
- read_media_file
- read_multiple_files
- write_file
- edit_file
- create_directory
- list_directory
- list_directory_with_sizes
- directory_tree
- move_file
- search_files
- get_file_info
- list_allowed_directories


### **4. Build Graph with MCP Tools**

We define the LangGraph structure, integrating tools fetched dynamically from the MCP server.

In [10]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_mcp_adapters.client import MultiServerMCPClient

async def build_graph_with_mcp_tools():
    client = MultiServerMCPClient(
        {
            "filesystem": {
                "command": "npx",
                "args": ["-y", "@modelcontextprotocol/server-filesystem", SANDBOX_DIR],
                "transport": "stdio",
            }
        }
    )

    tools = await client.get_tools()
    
    for t in tools:
        if isinstance(t.args_schema, dict) and "$schema" in t.args_schema:
            del t.args_schema["$schema"]


    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        google_api_key=os.environ["GOOGLE_API_KEY"],
    )

    llm_with_tools = llm.bind_tools(tools)

    async def chat(state: dict):
        response = await llm_with_tools.ainvoke(state["messages"])
        return {"messages": state["messages"] + [response]}

    tool_node = ToolNode(tools)

    g = StateGraph(dict)
    g.add_node("chatbot", chat)
    g.add_node("tools", tool_node)

    g.add_edge(START, "chatbot")
    g.add_conditional_edges("chatbot", tools_condition)
    g.add_edge("tools", "chatbot")

    return g.compile()


### **5. Execute and Verify**

Testing the agent by asking it to create and read a file in the sandbox.

In [11]:
app = await build_graph_with_mcp_tools()

result = await app.ainvoke({
    "messages": [
        HumanMessage(content="Create sandbox_dir/hello.txt with 'Hello I am raj gajjar' then read it.")
    ]
})

print(result["messages"][-1].content)


ValueError: contents are required.